# Develop, evaluate, and score a forecasting model for electricity generation

In [1]:
"""
 Introduction

In this notebook, you'll see Microsoft Fabric's end-to-end data science workflow for a forecasting model. This scenario uses the historic sales data to predict the sales for different categories of products at a superstore.

Forecasting is a crucial asset in sales, harnessing historical data and predictive methods to provide insights into future trends. By analyzing past sales, identifying patterns, and learning from consumer behavior, businesses can optimize inventory, production, and marketing strategies. This proactive approach enhances adaptability, responsiveness, and overall performance of businesses in a dynamic marketplace.

The main steps in this notebook are:

1. Load the data
2. Understand and process the data using exploratory data analysis
3. Train a machine learning model using an open source software package called `SARIMAX` and track experiments using MLflow and Fabric Autologging feature
4. Save the final machine learning model and make predictions
5. Demonstrate the model performance via visualizations in Power BI

Thesis

What percentage of fuel generation will come from Natural Gas in the coming months?

"""

StatementMeta(, b44d5d99-160a-4c42-b06b-f563272dffb5, 3, Finished, Available)

"\n Introduction\n\nIn this notebook, you'll see Microsoft Fabric's end-to-end data science workflow for a forecasting model. This scenario uses the historic sales data to predict the sales for different categories of products at a superstore.\n\nForecasting is a crucial asset in sales, harnessing historical data and predictive methods to provide insights into future trends. By analyzing past sales, identifying patterns, and learning from consumer behavior, businesses can optimize inventory, production, and marketing strategies. This proactive approach enhances adaptability, responsiveness, and overall performance of businesses in a dynamic marketplace.\n\nThe main steps in this notebook are:\n\n1. Load the data\n2. Understand and process the data using exploratory data analysis\n3. Train a machine learning model using an open source software package called `SARIMAX` and track experiments using MLflow and Fabric Autologging feature\n4. Save the final machine learning model and make pre

In [2]:
pip install pmdarima

StatementMeta(, b44d5d99-160a-4c42-b06b-f563272dffb5, 4, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.5 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


## Libraries and Variables

In [3]:
# Importing required libraries
import warnings
import itertools
# Record the notebook running time
import time
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
# Set up MLflow for experiment tracking
import mlflow
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import pmdarima as pm
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
# Import required libraries for model evaluation
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pyspark.sql.functions import col, to_date
from pmdarima.arima import auto_arima
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract
from statsmodels.stats.diagnostic import acorr_ljungbox
import os, requests
import statsmodels.api as sm
from mlflow import MlflowClient
from pprint import pprint
"""
`Statsmodels` is a Python module that provides classes and functions for the estimation of many different statistical models
It also allows you to conduct statistical tests and statistical data exploration.
"""

StatementMeta(, b44d5d99-160a-4c42-b06b-f563272dffb5, 5, Finished, Available)

'\n`Statsmodels` is a Python module that provides classes and functions for the estimation of many different statistical models\nIt also allows you to conduct statistical tests and statistical data exploration.\n'

In [4]:
"""
Define Variables
"""

EXPERIMENT_NAME = "EIA-forecasting-runs"  # MLflow experiment name

ts = time.time()

"""
ML Flow: Machine Learning

Autologging in Microsoft Fabric extends the MLflow autologging capabilities by automatically capturing the values of input parameters and output metrics of a machine learning model as it is being trained. This information is then logged to the workspace, where it can be accessed and visualized using the MLflow APIs or the corresponding experiment in the workspace. To learn more about autologging, see [Autologging in Microsoft Fabric](https://aka.ms/fabric-autologging).
"""

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.autolog(disable=True)  # Disable MLflow autologging



StatementMeta(, b44d5d99-160a-4c42-b06b-f563272dffb5, 6, Finished, Available)

## Step 1: Datasets

### Training Data

In [5]:
"""
Step 1: Read the dataset from the lakehouse
"""

df = spark.sql(
"""
SELECT 
Type.FuelClass Fuel_class
,Ops.date_full Operating_date
,sum(Fact.NetGenerationElectricityMWh) Electricity_generated
FROM EIA_Lake.fact_generation Fact
left outer join dim_fuel_type Type on Fact.FuelTypeKey = Type.FuelTypeKey
left outer join dim_date Ops on Fact.SurveyDateKey = Ops.date_id 
where Ops.date_full > '1999-12-31'
GROUP BY 
Type.FuelClass 
,Ops.date_full 
"""
)

Training_data = df

display(Training_data)

"""
Data Reformatting
"""
Training_data = Training_data.withColumnRenamed("Fuel_Class", "Fuel class")
Training_data = Training_data.withColumnRenamed("Operating_Date", "Operating date")
Training_data = Training_data.withColumnRenamed("Electricity_Generated", "Electricity generated")
Training_data = Training_data.withColumn("Operating date", to_date(col("Operating date")))
# Convert the PySpark DataFrame to a Pandas DataFrame
Training_data = Training_data.toPandas()

"""
How to specify or convert the data types post-conversion in the Pandas DataFrame
"""

# Assuming Training_data is your Pandas DataFrame
Training_data['Fuel class'] = Training_data['Fuel class'].astype(str)
# Assuming Training_data is your Pandas DataFrame
Training_data['Operating date'] = pd.to_datetime(Training_data['Operating date'])
# Convert the 'Electricity generated' column to int
Training_data['Electricity generated'] = Training_data['Electricity generated'].astype(int)

"""
The dataset is structured on a daily basis
The goal is to develop a model to forecast the sales on a monthly basis, you need to resample on the column `Order Date`.

1. group the `Furniture` category by `Order Date` and then 
2. calculate the sum of the `Sales` column for each group in order to determine the total sales for each unique `Order Date`. 
3. resample the `Sales` column using the `MS` frequency to aggregate the data by month and then you calculate the mean sales value for each month.
"""

# Data preprocessing
Training_data = Training_data.sort_values('Operating date')
Training_data.isnull().sum()
Training_data = Training_data.groupby('Operating date')['Electricity generated'].sum().reset_index()
Training_data = Training_data.set_index('Operating date')

"""
Add a docstring for exactly how this works. 
Explain interpolation for example. 
"""
# Assuming Historical_data is your DataFrame with a DateTime index
Training_data_resampled = Training_data['Electricity generated'].resample('MS').mean()
# Forward-fill missing values for the first few entries
Training_data_resampled_filled = Training_data_resampled.ffill()
# Interpolate missing values for the rest of the entries
Training_data_resampled_interpolated = Training_data_resampled_filled.interpolate(method='linear')
Training_data = Training_data_resampled_interpolated

Training_data = Training_data.fillna(0) # Fill NaN values with zero
Training_data = Training_data.astype('int64')
Training_data = Training_data.reset_index()
Training_data['Operating date'] = pd.to_datetime(Training_data['Operating date'])
maximim_date = Training_data.reset_index()['Operating date'].max()
Training_data = Training_data[Training_data['Operating date'].dt.strftime('%Y-%m-%d') < "2022-01-01"] # training data
# Training_data.sort_index(inplace=True)
Training_data = Training_data.set_index(['Operating date'])

print(Training_data)
display(maximim_date)
print(type(Training_data))
display(Training_data)

StatementMeta(, b44d5d99-160a-4c42-b06b-f563272dffb5, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2de5d4fb-55cb-4d30-a71c-21b1dc417595)

Empty DataFrame
Columns: [Electricity generated]
Index: []


Timestamp('2023-08-01 00:00:00')

<class 'pandas.core.frame.DataFrame'>


SynapseWidget(Synapse.DataFrame, f6e89c77-99d8-469f-bc76-a6e6a5a259bb)

### Validation Data

In [6]:
"""
Step 1: Read the dataset from the lakehouse
"""

df = spark.sql(
"""
SELECT 
Type.FuelClass Fuel_class
,Ops.date_full Operating_date
,sum(Fact.NetGenerationElectricityMWh) Electricity_generated
FROM EIA_Lake.fact_generation Fact
left outer join dim_fuel_type Type on Fact.FuelTypeKey = Type.FuelTypeKey
left outer join dim_date Ops on Fact.SurveyDateKey = Ops.date_id 
where Ops.date_full > '2021-12-31' and Ops.date_full < '2023-01-01'
GROUP BY 
Type.FuelClass 
,Ops.date_full 
"""
)

validation_data = df

"""
Data Reformatting
"""
validation_data = validation_data.withColumnRenamed("Fuel_Class", "Fuel class")
validation_data = validation_data.withColumnRenamed("Operating_Date", "Operating date")
validation_data = validation_data.withColumnRenamed("Electricity_Generated", "Electricity generated")
validation_data = validation_data.withColumn("Operating date", to_date(col("Operating date")))
# Convert the PySpark DataFrame to a Pandas DataFrame
validation_data = validation_data.toPandas()

# Type conversion is your Pandas DataFrame
validation_data['Fuel class'] = validation_data['Fuel class'].astype(str)
validation_data['Operating date'] = pd.to_datetime(validation_data['Operating date'])
validation_data['Electricity generated'] = validation_data['Electricity generated'].astype(int)

# Data preprocessing
validation_data = validation_data.sort_values('Operating date')
validation_data.isnull().sum()
validation_data = validation_data.groupby('Operating date')['Electricity generated'].sum().reset_index()
validation_data = validation_data.set_index('Operating date')

print(validation_data)
display(validation_data)

StatementMeta(, b44d5d99-160a-4c42-b06b-f563272dffb5, 8, Finished, Available)

Empty DataFrame
Columns: [Electricity generated]
Index: []


SynapseWidget(Synapse.DataFrame, 85089136-dbf6-4b2f-8e5a-2bc5a9e3242f)

## Step 2: Statistical Analysis

In [ ]:
"""
Statistical analysis

A time series tracks four data elements at set intervals in order to determine the variation of those four elements in the time series pattern. These elements include:

- **Level:** Refers to the fundamental component that represents the average value for a specific time period.

- **Trend:** Describes whether the time series is decreasing, constant, or increasing over time.

- **Seasonality:** Describes the periodic signal in the time series and looks for cyclic occurrences that affect the time series' increasing or decreasing patterns.

- **Noise/Residual:** Refers to the random fluctuations and variability in the time series data that cannot be explained by the model.

In the following, you will observe the above four components for your dataset after the pre-processing.
"""


# Decompose the time series into its components using statsmodels
result = sm.tsa.seasonal_decompose(Training_data, model='additive')

# Labels and corresponding data for plotting
components = [('Seasonality', result.seasonal),
              ('Trend', result.trend),
              ('Residual', result.resid),
              ('Observed Data', Training_data)]

# Create subplots in a grid
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(12, 7))
plt.subplots_adjust(hspace=0.8)  # Adjust vertical space
axes = axes.ravel()

# Plot the components
for ax, (label, data) in zip(axes, components):
    ax.plot(data, label=label, color='blue' if label != 'Observed Data' else 'purple')
    ax.set_xlabel('Time')
    ax.set_ylabel(label)
    ax.set_xlabel('Time', fontsize=10)
    ax.set_ylabel(label, fontsize=10)
    ax.legend(fontsize=10)

plt.show()

"""
Doing time series analysis. We are preforming seasonal decomposition. This gives an overall understanding of how these components contribute to the entire time series.
Understanding the seasonality, trend, and noise in the forecasting data through the above plots allows to capture underlying patterns, and develop models that make more accurate predictions that are resilient to random fluctuations.
"""

In [ ]:
"""
Statistical Measure of accuracy
"""
def mean_absolute_error(y_true, y_pred):
    """
    Calculate Mean Absolute Error (MAE)
    
    Args:
    y_true: List or array containing true values
    y_pred: List or array containing predicted values
    
    Returns:
    mae: Mean Absolute Error
    """
    if len(y_true) != len(y_pred):
        raise ValueError("Lengths of y_true and y_pred must be the same.")
    
    total_error = 0
    for true_val, pred_val in zip(y_true, y_pred):
        total_error += abs(true_val - pred_val)
    
    mae = total_error / len(y_true)
    return mae

## Step 3: Model Training and Tracking

With your data in place, you can define the forecasting model. Apply the Seasonal AutoRegressive Integrated Moving Average with eXogenous regressors (SARIMAX) in this notebook. SARIMAX is a time series forecasting model that extends SARIMA to include exogenous variables. It combines autoregressive (AR) and moving average (MA) components, seasonal differencing, and external predictors to make accurate and flexible forecasts for time series data, making it a powerful tool for various forecasting tasks.

You will also use MLfLow and Fabric Autologging to track the experiments. Here you'll load the delta table from the lakehouse. You may use other delta tables considering the lakehouse as the source.

"""
SARIMAX, or Seasonal Autoregressive Integrated Moving Average with Exogenous Factors, 
a time series forecasting method that extends the traditional ARIMA model to include seasonal components and exogenous variables. 

The acronym breaks down as follows:
- Seasonal: Accounts for patterns that repeat over known intervals or seasons.
- ARIMA: Stands for Autoregressive Integrated Moving Average, a model used for understanding and forecasting time series data.
- X: Denotes the inclusion of exogenous variables, which are external factors that can influence the time series being forecasted.

In SARIMAX models, the seasonal and non-seasonal components of the time series data are modeled separately. 
It's a powerful tool for handling time series data that exhibits both seasonal patterns and relationships with other variables.
It's often used
"""

### Hyperparameter tuning

SARIMAX takes into account the parameters involved in regular ARIMA mode `(p,d,q)` and also adds the seasonality parameters `(P,D,Q,s)`. These arguments to SARIMAX model are called order `(p,d,q)` and seasonal order `(P,D,Q,s)` respectively and hence 7 parameters to tune. Prior to model training, you need to set up these parameters which are defined in the following.

#### Order Parameters `(p, d, q)`:
- `p`: The order of the autoregressive (AR) component, indicating how many past observations are considered. It is also known as the AR order.
- `d`: The degree of differencing required to make the time series stationary. It is also known as the differencing order.
- `q`: The order of the moving average (MA) component, indicating how many past white noise error terms are considered. It is also known as the MA order.

#### Seasonal Order Parameters `(P, D, Q, s)`:

- `P`: The seasonal order of the autoregressive (AR) component, similar to `p` but for the seasonal part.
- `D`: The seasonal order of differencing, similar to `d` but for the seasonal part.
- `Q`: The seasonal order of the moving average (MA) component, similar to `q` but for the seasonal part.
- `s`: The number of time steps per seasonal cycle (e.g., 12 for monthly data with a yearly seasonality).


 The autoregressive order `p` represents the number of past observations in the time series that are used to predict the current value. Typically, `p` should be a non-negative integer. Common values for `p` are usually in the range of 0 to 3, although higher values are possible depending on the specific characteristics of the data. A higher p indicates a longer memory of past values in the model.

 The moving average order `q` represents the number of past white noise error terms that are used to predict the current value. Similar to `p`, `q` should also be a non-negative integer. Common values for `q` are typically in the range of 0 to 3, but higher values may be necessary for certain time series. A higher `q` indicates a stronger reliance on past error terms to make predictions.

 The differencing order `d` represents the number of times the time series needs to be differenced to achieve stationarity. `d` should be a non-negative integer. Common values for `d` are usually in the range of 0 to 2. A `d` value of 0 means the time series is already stationary, while higher values indicate the number of differencing operations required to make it stationary.


#### Determine Optional Parameters:
- The `enforce_stationarity` parameter controls whether or not the model should enforce stationarity on the time series data before fitting the SARIMAX model. When `enforce_stationarity` is set to `True` (the default), it indicates that the SARIMAX model should enforce stationarity on the time series data. This means that the SARIMAX model will automatically apply differencing to the data to make it stationary, as specified by the `d` and `D` orders, before fitting the model. This is a common practice because many time series models, including SARIMAX, assume that the data is stationary. If your time series is non-stationary (e.g., it exhibits trends or seasonality), it is generally a good practice to set `enforce_stationarity` to `True` and let the SARIMAX model handle the differencing to achieve stationarity. If your time series is already stationary (e.g., it has no trends or seasonality), you can set `enforce_stationarity` to `False` to avoid unnecessary differencing.

- The `enforce_invertibility` parameter controls whether or not the model should enforce invertibility on the estimated parameters during the optimization process. When `enforce_invertibility` is set to `True` (the default), it indicates that the SARIMAX model should enforce invertibility on the estimated parameters. Invertibility ensures that the model is well-defined and that the estimated autoregressive (AR) and moving average (MA) coefficients are within the range of stationarity. Enforcing invertibility is typically recommended to ensure that the SARIMAX model adheres to the theoretical requirements for a stable time series model and helps prevent issues with model estimation and stability.

- The default is an `AR(1)` model which refers to `(1,0,0)`. However, keep in mind that the appropriate values for `p`, `q`, and `d` can vary from one time series to another, and determining the optimal values often involves analyzing the autocorrelation and partial autocorrelation functions (ACF and PACF) of the time series data and using model selection criteria like AIC or BIC. It's common practice to try different combinations of p, q, and d and evaluate the model's performance for a given dataset. Note that the parameters for the seasonal order `(P, D, Q, s)` is similar in concept to the non-seasonal order parameters `(p, q, d)`, hence it is avoided explaining in detail again. 

In [ ]:
def fetch_logged_data(run_id):
    client = MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    # artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags

In [ ]:
def check_stationarity(data):
    result = adfuller(data)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value}')
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("Weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

# Call the function with your time series data 'y'
check_stationarity(Training_data)


#### Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) Method

##### Find Difference Values

In [ ]:

# Original time series data
original_data = Training_data['Electricity generated']

# First-order differences
first_differences = [original_data[i] - original_data[i - 1] for i in range(1, len(original_data))]

# Second-order differences
second_differences = [original_data[i] - original_data[i - 2] for i in range(2, len(original_data))]

# Third-order differences
third_differences = [original_data[i] - original_data[i - 3] for i in range(3, len(original_data))]

# Calculating means for each differenced series
first_diff_mean = np.mean(first_differences)
second_diff_mean = np.mean(second_differences)
third_diff_mean = np.mean(third_differences)

# Plotting the original data
plt.figure(figsize=(18, 6))

# Plotting the original data
plt.subplot(141)
plt.plot(original_data, marker='o', color='blue')
plt.title('Original Time Series')
plt.xlabel('Index')
plt.ylabel('Value')

# Plotting the first-order differences
plt.subplot(142)
plt.plot(first_differences, marker='o', color='red')
plt.axhline(y=first_diff_mean, color='black', linestyle='--', label=f'Mean: {first_diff_mean:.2f}')
plt.title('First-order Differences (d=1)')
plt.xlabel('Index')
plt.ylabel('Differences')
plt.legend()

# Plotting the second-order differences
plt.subplot(143)
plt.plot(second_differences, marker='o', color='green')
plt.axhline(y=second_diff_mean, color='black', linestyle='--', label=f'Mean: {second_diff_mean:.2f}')
plt.title('Second-order Differences (d=2)')
plt.xlabel('Index')
plt.ylabel('Differences')
plt.legend()

# Plotting the third-order differences
plt.subplot(144)
plt.plot(third_differences, marker='o', color='orange')
plt.axhline(y=third_diff_mean, color='black', linestyle='--', label=f'Mean: {third_diff_mean:.2f}')
plt.title('Third-order Differences (d=3)')
plt.xlabel('Index')
plt.ylabel('Differences')
plt.legend()
plt.show()




In [ ]:
# Assuming 'first_differences', 'second_differences', 'third_differences' contain the respective differenced series

# Create timestamp indices for the differenced series (assuming monthly data)
start_date = pd.to_datetime('2000-03-01')
end_date = pd.to_datetime('2021-12-01')  # Adjust based on your data length
timestamp1 = pd.date_range(start=pd.to_datetime('2000-01-01'), end=end_date, freq='M')
timestamp2 = pd.date_range(start=pd.to_datetime('2000-02-01'), end=end_date, freq='M')
timestamp3 = pd.date_range(start=pd.to_datetime('2000-03-01'), end=end_date, freq='M')
# Create pandas Series with timestamps for each differenced series
differenced_series1 = pd.Series(first_differences, index=timestamp1)
differenced_series2 = pd.Series(second_differences, index=timestamp2)
differenced_series3 = pd.Series(third_differences, index=timestamp3)

# Calculating means for each differenced series
first_diff_mean = np.mean(differenced_series1)
second_diff_mean = np.mean(differenced_series2)
third_diff_mean = np.mean(differenced_series3)

# Decompose each time series into its components using statsmodels
result1 = sm.tsa.seasonal_decompose(differenced_series1, model='additive')
result2 = sm.tsa.seasonal_decompose(differenced_series2, model='additive')
result3 = sm.tsa.seasonal_decompose(differenced_series3, model='additive')

# Labels and corresponding data for plotting for each series
components1 = [('Seasonality', result1.seasonal),
               ('Trend', result1.trend),
               ('Residual', result1.resid),
               ('Observed Data', differenced_series1)]

components2 = [('Seasonality', result2.seasonal),
               ('Trend', result2.trend),
               ('Residual', result2.resid),
               ('Observed Data', differenced_series2)]

components3 = [('Seasonality', result3.seasonal),
               ('Trend', result3.trend),
               ('Residual', result3.resid),
               ('Observed Data', differenced_series3)]

# Create subplots in a grid for each series
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(15, 10))
plt.subplots_adjust(hspace=0.8)  # Adjust vertical space
axes = axes.ravel()

# Plot the components for each series
for i, (label, data) in enumerate(components1):
    ax = axes[i]
    ax.plot(data, label=label, color='blue' if label != 'Observed Data' else 'purple')
    ax.axhline(y=first_diff_mean, color='black', linestyle='--', label=f'Mean: {first_diff_mean:.2f}')
    ax.set_xlabel('Time')
    ax.set_ylabel(label)
    ax.set_xlabel('Time', fontsize=10)
    ax.set_ylabel(label, fontsize=10)
    ax.legend(fontsize=8)

for i, (label, data) in enumerate(components2):
    ax = axes[i + 4]
    ax.plot(data, label=label, color='green' if label != 'Observed Data' else 'purple')
    ax.axhline(y=second_diff_mean, color='black', linestyle='--', label=f'Mean: {second_diff_mean:.2f}')
    ax.set_xlabel('Time')
    ax.set_ylabel(label)
    ax.set_xlabel('Time', fontsize=10)
    ax.set_ylabel(label, fontsize=10)
    ax.legend(fontsize=8)

for i, (label, data) in enumerate(components3):
    ax = axes[i + 8]
    ax.plot(data, label=label, color='orange' if label != 'Observed Data' else 'purple')
    ax.axhline(y=third_diff_mean, color='black', linestyle='--', label=f'Mean: {third_diff_mean:.2f}')
    ax.set_xlabel('Time')
    ax.set_ylabel(label)
    ax.set_xlabel('Time', fontsize=10)
    ax.set_ylabel(label, fontsize=10)
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()


In [ ]:
check_stationarity(differenced_series1)

#### Grid search method.

In [ ]:
mlflow.autolog(disable=True)  # Disable MLflow autologging

# Grid search method.
"""
The auto_arima function itself operates a bit like a grid search, 
in that it tries various sets of p and q (also P and Q for seasonal models) parameters, 
selecting the model that minimizes the AIC (or BIC, or whatever information criterion you select). 
To select the differencing terms, auto_arima uses a test of stationarity (such as an augmented Dickey-Fuller test)
 and seasonality (such as the Canova-Hansen test) for seasonal models.
 """



# fit stepwise auto-ARIMA
stepwise_fit = pm.auto_arima(Training_data, start_p=1, start_q=1,
                             max_p=7, max_q=7, m=12,
                             start_P=0, seasonal=True,
                             d=1, D=1, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise


# After obtaining stepwise_fit using auto_arima

# Accessing the summary() method to get the best model summary
best_model_summary = stepwise_fit.summary()

spark = SparkSession.builder.appName("ARIMA_Parameters").getOrCreate()

# Extracting the best model string from the summary
best_model_str = str(best_model_summary)
best_model_str = best_model_str.split('\n')[3].split(':')[1].strip()

print(best_model_str)

# # Define the pattern for extracting ARIMA parameters
# pattern = r'ARIMA\((\d+),(\d+),(\d+)\)\((\d+),(\d+),(\d+)\)\[(\d+)\]'

# Define the pattern for extracting ARIMA parameters
pattern = r'SARIMAX\((\d+), (\d+), (\d+)\)x\((\d+), (\d+), \[(\d+)\], (\d+)\)'


# Using regexp_extract to extract the values
extracted_values = spark.createDataFrame([(best_model_str,)], ['model_string']).select(
    regexp_extract('model_string', pattern, 1).alias('p'),
    regexp_extract('model_string', pattern, 2).alias('d'),
    regexp_extract('model_string', pattern, 3).alias('q'),
    regexp_extract('model_string', pattern, 4).alias('P'),
    regexp_extract('model_string', pattern, 5).alias('D'),
    regexp_extract('model_string', pattern, 6).alias('Q'),
    regexp_extract('model_string', pattern, 7).alias('s')
)

# Extracted values as strings
extracted_row = extracted_values.collect()[0]
print(extracted_values)
print(extracted_row)
p_str = extracted_row['p']
d_str = extracted_row['d']
q_str = extracted_row['q']
P_str = extracted_row['P']
D_str = extracted_row['D']
Q_str = extracted_row['Q']
s_str = extracted_row['s']

# Convert the extracted strings to integers with error handling
def convert_to_int(val):
    try:
        return int(val) if val.strip() else None
    except ValueError:
        return None

# Conversion with error handling
p = convert_to_int(p_str)
d = convert_to_int(d_str)
q = convert_to_int(q_str)
P = convert_to_int(P_str)
D = convert_to_int(D_str)
Q = convert_to_int(Q_str)
s = convert_to_int(s_str)

# Check for negative differencing and non-integer values
if d is not None and (not isinstance(d, int) or d < 0):
    raise ValueError('Differencing must be a non-negative integer.')


mlflow.autolog(disable=False)  # Enable MLflow autologging

# Start an MLflow run
with mlflow.start_run():

    mod = sm.tsa.statespace.SARIMAX(Training_data,
                                order=(p, d, q),
                                seasonal_order=(P, D, Q, s),
                                enforce_stationarity=True,
                                enforce_invertibility=True)
    results = mod.fit(disp=False)
    # Forecasting on the validation dataset (2022)
    forecast = results.get_forecast(steps=len(validation_data))

    # Compare forecast to actuals for evaluation
    forecast_values = forecast.predicted_mean

    # Log SARIMAX model parameters
    mlflow.log_params({"order": (p, d, q), "seasonal_order": (P, D, Q, s), 'enforce_stationarity': True, 'enforce_invertibility': True})


# Log the model and parameters
model_name = f"{EXPERIMENT_NAME}-Sarimax"
with mlflow.start_run(run_name="Sarimax") as run:
    mlflow.statsmodels.log_model(results,model_name,registered_model_name=model_name)
    mlflow.log_params({"order":(p,d,q),"seasonal_order":(P, D, Q, s),'enforce_stationarity':True,'enforce_invertibility':True})
    model_uri = f"runs:/{run.info.run_id}/{model_name}"
    print("Model saved in run %s" % run.info.run_id)
    print(f"Model URI: {model_uri}")
mlflow.end_run()

# Load the saved model
loaded_model = mlflow.statsmodels.load_model(model_uri)

# Validation Steps
Future = pd.DataFrame(forecast_values).reset_index()
validation = pd.DataFrame(validation_data).reset_index()
Future['Operating date'] = pd.to_datetime(Future['index'])
# Using merge
result = pd.merge(validation, Future[['Operating date', 'predicted_mean']], on='Operating date', how='left')
result['Order'] = result.apply(lambda row: f"SARIMAX({p}, {d}, {q})x({P}, {D}, {Q}, {s})", axis=1)
result['MAPE'] = np.NAN
# Calculate the Mean Absolute Percentage Error (MAPE) between the 'Actual_Sales' and 'Forecasted_Sales' 
result['MAPE'] = mean_absolute_percentage_error(result['Electricity generated'], result['predicted_mean']) * 100
result['MAE'] = mean_absolute_error(result['Electricity generated'], result['predicted_mean'])
result = result.reindex(columns=['Order','Operating date', 'Electricity generated', 'predicted_mean', 'MAPE', 'MAE'])
display(result)


In [ ]:
result.rename(columns={'Operating date': 'Operating_Date'}, inplace=True)
result.rename(columns={'Electricity generated': 'Electricity_generated'}, inplace=True)
result.rename(columns={'predicted_mean': 'Forecast_Electricity'}, inplace=True)

# Write Back the results into the lakehouse
table_name = "Grid_Forecast"
spark.createDataFrame(result).write.mode("overwrite").format("delta").save(f"Tables/{table_name}")
print(f"Spark dataframe saved to delta table: {table_name}")

#### Best Fit method (Lowest MAPE & MAE)

In [ ]:
p = d = q = range(0, 8)

# Non-seasonal combinations
pdq_combinations = list(itertools.product(p, d, q))
num_pdq_combinations = len(pdq_combinations)

# Seasonal combinations
seasonal_pdq_combinations = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
num_seasonal_pdq_combinations = len(seasonal_pdq_combinations)

# Total combinations
total_combinations = num_pdq_combinations * num_seasonal_pdq_combinations

print(f"Number of pdq combinations: {num_pdq_combinations}")
print(f"Number of seasonal_pdq combinations: {num_seasonal_pdq_combinations}")
print(f"Total number of combinations: {total_combinations}")
print("need to optimize the search process or utilizing parallel computing can significantly reduce the time taken to find the best parameters for the model.")


In [ ]:
validation_data['Electricity generated'] = pd.to_numeric(validation_data['Electricity generated'], errors='coerce')

In [ ]:
class ModelCombination:
    def __init__(self, param, param_seasonal):
        self.param = param
        self.param_seasonal = param_seasonal
        # self.aic = aic
        self.model = None

def find_best_model(y):
    best_aic = np.inf
    best_params = None
    best_model = None
    lowest_mape = np.inf  # Initialize with a high value
    best_model_info = None  # Initialize as None
    # Define ranges for hyperparameters
    p = d = q = range(0, 5)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
    model_name = f"{EXPERIMENT_NAME}-Sarimax"

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                with mlflow.start_run():
                    mod = sm.tsa.statespace.SARIMAX(Training_data,
                                                    order=param,
                                                    seasonal_order=param_seasonal,
                                                    enforce_stationarity=True,
                                                    enforce_invertibility=True)
                    results = mod.fit(disp=False)
                    # Forecasting on the validation dataset (2022)
                    forecast = results.get_forecast(steps=len(validation_data))

                    # Compare forecast to actuals for evaluation
                    forecast_values = forecast.predicted_mean

                    best_params = (param, param_seasonal)
                    best_model = ModelCombination(param, param_seasonal)
                    best_model.model = results  # Storing the best model object

                    # Log SARIMAX model parameters
                    mlflow.log_params({"order": best_model.param, "seasonal_order": best_model.param_seasonal, 'enforce_stationarity': True, 'enforce_invertibility': True})
                    mape = mean_absolute_percentage_error(validation_data['Electricity generated'], forecast_values) * 100
                    mae = mean_absolute_error(validation_data['Electricity generated'], forecast_values)

                    # Check if the current MAPE is lower than the lowest recorded
                    if mape < lowest_mape:
                        lowest_mape = mape  # Update the lowest MAPE
                        best_model_info = {
                            'param': param,
                            'param_seasonal': param_seasonal,
                            'mape': lowest_mape,
                            'model_summary': best_model_str  # or any other relevant model info
                        }


                    mlflow.log_metric("MAPE", mape)
                    mlflow.log_metric("MAE", mae)

                with mlflow.start_run(run_name="Sarimax") as run:
                    mlflow.statsmodels.log_model(results,model_name,registered_model_name=model_name)
                    # mlflow.statsmodels.log_model(best_model.model, model_name, registered_model_name=model_name)
                    mlflow.log_params({"order": best_model.param, "seasonal_order": best_model.param_seasonal, 'enforce_stationarity': True, 'enforce_invertibility': True})
                    mlflow.log_metric("MAPE", mape)
                    mlflow.log_metric("MAE", mae)               
                    model_uri = f"runs:/{run.info.run_id}/{model_name}"
                    print("Best model saved in run %s" % run.info.run_id)
                    print(f"Best model URI: {model_uri}")
                    mlflow.end_run()
                # Load the saved model
                loaded_model = mlflow.statsmodels.load_model(model_uri)

                # Validation Steps
                Future = pd.DataFrame(forecast_values).reset_index()
                validation = pd.DataFrame(validation_data).reset_index()
                Future['Operating date'] = pd.to_datetime(Future['index'])
                # Using merge
                result = pd.merge(validation, Future[['Operating date', 'predicted_mean']], on='Operating date', how='left')

                # Accessing the summary() method to get the best model summary
                best_model_summary = results.summary()



                # Extracting the best model string from the summary
                best_model_str = str(best_model_summary)
                best_model_str = best_model_str.split('\n')[3].split(':')[1].strip()

         
                result['Order'] = best_model_str
                result['MAPE'] = np.NAN
                # Calculate the Mean Absolute Percentage Error (MAPE) between the 'Actual_Sales' and 'Forecasted_Sales' 
                result['MAPE'] = mean_absolute_percentage_error(result['Electricity generated'], result['predicted_mean']) * 100
                result['MAE'] = mean_absolute_error(result['Electricity generated'], result['predicted_mean'])


                result = result.reindex(columns=['Order','Operating date', 'Electricity generated', 'predicted_mean', 'MAPE', 'MAE'])
                display(result)


                result.rename(columns={'Operating date': 'Operating_Date'}, inplace=True)
                result.rename(columns={'Electricity generated': 'Electricity_generated'}, inplace=True)
                result.rename(columns={'predicted_mean': 'Forecast_Electricity'}, inplace=True)

                # Write Back the results into the lakehouse
                table_name = "EIA_Test"
                spark.createDataFrame(result).write.mode("append").format("delta").save(f"Tables/{table_name}")
                print(f"Spark dataframe saved to delta table: {table_name}")
            except Exception as e:
                print(f"Model fitting failed for {param}, {param_seasonal}. Error: {str(e)}")
                continue 
                           
    # After all iterations are done, you can access the best model information
    if best_model_info is not None:
        print("Best model found with lowest MAPE:")
        print(f"Parameters: {best_model_info['param']}")
        print(f"Seasonal Parameters: {best_model_info['param_seasonal']}")
        print(f"Lowest MAPE: {best_model_info['mape']}")
        print(f"Model Summary: {best_model_info['model_summary']}")
    else:
        print("No valid model found")



In [ ]:
find_best_model(Training_data)

#### AIC (Akaike Information Criterion) method

In [ ]:
# Best version of AIC

class ModelCombination:
    def __init__(self, param, param_seasonal, aic):
        self.param = param
        self.param_seasonal = param_seasonal
        self.aic = aic
        self.model = None

def find_best_model(y):
    best_aic = np.inf
    best_params = None
    best_model = None

    # Define ranges for hyperparameters
    p = d = q = range(0, 8)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
    model_name = f"{EXPERIMENT_NAME}-Sarimax"

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(y,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=True,
                                                enforce_invertibility=True)
                results = mod.fit(disp=False)
                aic = results.aic
 
                


                # Check if current AIC is the best so far
                if aic < best_aic:
                    best_aic = aic
                    best_params = (param, param_seasonal)
                    best_model = ModelCombination(param, param_seasonal, aic)
                    best_model.model = results  # Storing the best model object
            except Exception as e:
                print(f"Model fitting failed for {param}, {param_seasonal}. Error: {str(e)}")
                continue

    # Log the best model and parameters using MLflow
    if best_model is not None and best_model.model is not None:
        with mlflow.start_run(run_name="Sarimax") as run:
            mlflow.statsmodels.log_model(best_model.model, model_name, registered_model_name=model_name)
            mlflow.log_params({"order": best_model.param, "seasonal_order": best_model.param_seasonal, 'enforce_stationarity': True, 'enforce_invertibility': True})
            model_uri = f"runs:/{run.info.run_id}/{model_name}"
            print("Best model saved in run %s" % run.info.run_id)
            print(f"Best model URI: {model_uri}")
            mlflow.end_run()
            return model_uri, best_model.model  
    else:
        print("No valid model found")




In [ ]:
"""
# I need this part
model_uri, overall_results = find_best_model(Training_data)
model_uri = model_uri
results = overall_results
print(model_uri)
print(results.summary())
# Load the saved model
loaded_model = mlflow.statsmodels.load_model(model_uri)
"""


#### Training Method I used (Manual)

In [ ]:
p = 7
d = 1
q = 5
P = 1
D = 1
Q = 1
s = 12

# Start an MLflow run
with mlflow.start_run():

    mod = sm.tsa.statespace.SARIMAX(Training_data,
                                order=(p, d, q),
                                seasonal_order=(P, D, Q, s),
                                enforce_stationarity=True,
                                enforce_invertibility=True)
    results = mod.fit(disp=False)
    # Forecasting on the validation dataset (2022)
    forecast = results.get_forecast(steps=len(validation_data))

    # Compare forecast to actuals for evaluation
    forecast_values = forecast.predicted_mean

    # Log SARIMAX model parameters
    mlflow.log_params({"order": (p, d, q), "seasonal_order": (P, D, Q, s), 'enforce_stationarity': True, 'enforce_invertibility': True})


# Log the model and parameters
model_name = f"{EXPERIMENT_NAME}-Sarimax"
with mlflow.start_run(run_name="Sarimax") as run:
    mlflow.statsmodels.log_model(results,model_name,registered_model_name=model_name)
    mlflow.log_params({"order":(p,d,q),"seasonal_order":(P, D, Q, s),'enforce_stationarity':True,'enforce_invertibility':True})
    model_uri = f"runs:/{run.info.run_id}/{model_name}"
    print("Model saved in run %s" % run.info.run_id)
    print(f"Model URI: {model_uri}")
mlflow.end_run()

# Load the saved model
loaded_model = mlflow.statsmodels.load_model(model_uri)

# fetch logged data
params, metrics, tags = fetch_logged_data(run.info.run_id)

pprint(params)

pprint(metrics)

pprint(tags)

# pprint(artifacts)

# Validation Steps
Future = pd.DataFrame(forecast_values).reset_index()
validation = pd.DataFrame(validation_data).reset_index()
Future['Operating date'] = pd.to_datetime(Future['index'])
# Using merge
result = pd.merge(validation, Future[['Operating date', 'predicted_mean']], on='Operating date', how='left')
result['Order'] = result.apply(lambda row: f"SARIMAX({p}, {d}, {q})x({P}, {D}, {Q}, {s})", axis=1)
result['MAPE'] = np.NAN
# Calculate the Mean Absolute Percentage Error (MAPE) between the 'Actual_Sales' and 'Forecasted_Sales' 
result['MAPE'] = mean_absolute_percentage_error(result['Electricity generated'], result['predicted_mean']) * 100
result['MAE'] = mean_absolute_error(result['Electricity generated'], result['predicted_mean'])
result = result.reindex(columns=['Order','Operating date', 'Electricity generated', 'predicted_mean', 'MAPE', 'MAE'])
display(result)


result.rename(columns={'Operating date': 'Operating_Date'}, inplace=True)
result.rename(columns={'Electricity generated': 'Electricity_generated'}, inplace=True)
result.rename(columns={'predicted_mean': 'Forecast_Electricity'}, inplace=True)

# Write Back the results into the lakehouse
table_name = "EIA_Forecast"
spark.createDataFrame(result).write.mode("overwrite").format("delta").save(f"Tables/{table_name}")
print(f"Spark dataframe saved to delta table: {table_name}")

## Step 4: Score the model and save predictions

Scoring a model typically refers to evaluating its performance or assessing how well it predicts or fits the data it was trained on or applied to. For instance, in the context of a SARIMAX (Seasonal Autoregressive Integrated Moving Average with Exogenous Factors) model, scoring could involve several evaluation metrics.

Here are a few common ways to score a SARIMAX model:

1. **Mean Squared Error (MSE):** Measures the average of the squared differences between predicted and actual values. Lower MSE indicates better performance.

2. **Root Mean Squared Error (RMSE):** The square root of the MSE, giving an error value in the same units as the target variable. Lower RMSE signifies better performance.

3. **Mean Absolute Error (MAE):** Measures the average of the absolute differences between predicted and actual values. Similar to MSE but less sensitive to outliers.

4. **AIC (Akaike Information Criterion) or BIC (Bayesian Information Criterion):** These are used for model selection among a set of models. Lower AIC or BIC values indicate a better fit, with a balance between goodness of fit and model complexity.

5. **R-squared (R²) or adjusted R-squared:** Measures the proportion of variance in the dependent variable that is predictable from the independent variables. Higher R² values indicate a better fit of the model to the data.

When applying a SARIMAX model, you'd use historical data for training, leaving out a portion (the validation or test set) to evaluate its predictive performance. After fitting the model on the training data, you'd use the test set to generate predictions and then compare these predictions to the actual values to calculate these scoring metrics.

These scores help to gauge how well the model is performing and whether it needs adjustments or fine-tuning to improve its predictions.

### Re-training and Prediction

Re-training: Once validated, retrain the model on the entire dataset (training + validation) to utilize all available data.
Prediction: Use the retrained model to forecast future time points beyond the last known data point.

In [ ]:
# df = spark.sql(
# """
# SELECT 
# Type.FuelClass Fuel_class
# ,Ops.date_full Operating_date
# ,sum(Fact.NetGenerationElectricityMWh) Electricity_generated
# FROM EIA_Lake.fact_generation Fact
# left outer join dim_fuel_type Type on Fact.FuelTypeKey = Type.FuelTypeKey
# left outer join dim_date Ops on Fact.SurveyDateKey = Ops.date_id 
# where Ops.date_full > '1999-12-31'
# GROUP BY 
# Type.FuelClass 
# ,Ops.date_full 
# """
# )

# Historical_data = df

# """
# Data Reformatting
# """
# Historical_data = Historical_data.withColumnRenamed("Fuel_Class", "Fuel class")
# Historical_data = Historical_data.withColumnRenamed("Operating_Date", "Operating date")
# Historical_data = Historical_data.withColumnRenamed("Electricity_Generated", "Electricity generated")
# Historical_data = Historical_data.withColumn("Operating date", to_date(col("Operating date")))
# # Convert the PySpark DataFrame to a Pandas DataFrame
# Historical_data = Historical_data.toPandas()

# """
# How to specify or convert the data types post-conversion in the Pandas DataFrame
# """

# # Assuming Historical_data is your Pandas DataFrame
# Historical_data['Fuel class'] = Historical_data['Fuel class'].astype(str)
# # Assuming Historical_data is your Pandas DataFrame
# Historical_data['Operating date'] = pd.to_datetime(Historical_data['Operating date'])
# # Convert the 'Electricity generated' column to int
# Historical_data['Electricity generated'] = Historical_data['Electricity generated'].astype(int)

# """
# The dataset is structured on a daily basis
# The goal is to develop a model to forecast the sales on a monthly basis, you need to resample on the column `Order Date`.

# 1. group the `Furniture` category by `Order Date` and then 
# 2. calculate the sum of the `Sales` column for each group in order to determine the total sales for each unique `Order Date`. 
# 3. resample the `Sales` column using the `MS` frequency to aggregate the data by month and then you calculate the mean sales value for each month.
# """

# # Data preprocessing
# Historical_data = Historical_data.sort_values('Operating date')
# Historical_data.isnull().sum()
# Historical_data = Historical_data.groupby('Operating date')['Electricity generated'].sum().reset_index()
# Historical_data = Historical_data.set_index('Operating date')

# # Assuming Historical_data is your DataFrame with a DateTime index
# Historical_data_resampled = Historical_data['Electricity generated'].resample('MS').mean()
# # Forward-fill missing values for the first few entries
# Historical_data_resampled_filled = Historical_data_resampled.ffill()
# # Interpolate missing values for the rest of the entries
# Historical_data_resampled_interpolated = Historical_data_resampled_filled.interpolate(method='linear')
# Historical_data = Historical_data_resampled_interpolated
# Historical_data = Historical_data.fillna(0)  # Fill NaN values with zero
# Historical_data = Historical_data.astype('int64')
# Historical_data = Historical_data.reset_index()
# Historical_data['Operating date'] = pd.to_datetime(Historical_data['Operating date'])
# # Historical_data.sort_index(inplace=True)
# Historical_data = Historical_data.set_index(['Operating date'])

# p = 7
# d = 1
# q = 5
# P = 1
# D = 1
# Q = 1
# s = 12
# # Log the model and parameters
# model_name = f"{EXPERIMENT_NAME}-Sarimax"

# # Start an MLflow run
# with mlflow.start_run():

#     mod = sm.tsa.statespace.SARIMAX(Training_data, order=(p, d, q), seasonal_order=(P, D, Q, s),enforce_stationarity=True,enforce_invertibility=True)

#     results = mod.fit(disp=False)
#     # This initiates the forecast one month after maximim_date and for the next 6 months (months=6)
#     forecast = results.get_prediction(start=maximim_date + pd.DateOffset(months=1), end=maximim_date + pd.DateOffset(months=12), dynamic=False)

#     # Compare forecast to actuals for evaluation
#     forecast_values = forecast.predicted_mean

#     # Log SARIMAX model parameters
#     mlflow.log_params({"order": (p, d, q), "seasonal_order": (P, D, Q, s), 'enforce_stationarity': True, 'enforce_invertibility': True})


# with mlflow.start_run(run_name="Sarimax") as run:
#     mlflow.statsmodels.log_model(results, model_name, registered_model_name=model_name)
#     mlflow.log_params({"order": (p, d, q), "seasonal_order": (P, D, Q, s), 'enforce_stationarity': True,'enforce_invertibility': True})
#     model_uri = f"runs:/{run.info.run_id}/{model_name}"
#     print("Model saved in run %s" % run.info.run_id)
#     print(f"Model URI: {model_uri}")
# mlflow.end_run()

# # Load the saved model
# loaded_model = mlflow.statsmodels.load_model(model_uri)


In [ ]:

# # Validation Steps
# Future = pd.DataFrame(forecast_values).reset_index()
# Historical_data = pd.DataFrame(Historical_data).reset_index()
# Future['Operating date'] = pd.to_datetime(Future['index'])

# Historical_data['Order'] = Future.apply(lambda row: f"SARIMAX({p}, {d}, {q})x({P}, {D}, {Q}, {s})", axis=1)
# Historical_data['MAPE'] = np.NAN
# Historical_data['MAE'] = np.NAN
# Future['Order'] = Future.apply(lambda row: f"SARIMAX({p}, {d}, {q})x({P}, {D}, {Q}, {s})", axis=1)
# Future['MAPE'] = np.NAN
# Future['MAE'] = np.NAN

# Historical_data.rename(columns={'Electricity generated': 'Electricity_generated'}, inplace=True)
# Historical_data.rename(columns={'Operating date': 'Operating_Date'}, inplace=True)
# Historical_data = Historical_data.reindex(columns=['Order','Operating_Date', 'Electricity_generated', 'MAPE', 'MAE'])
# Future.rename(columns={'Operating date': 'Operating_Date'}, inplace=True)
# # Future.rename(columns={'Electricity generated': 'Electricity_generated'}, inplace=True)
# Future.rename(columns={'predicted_mean': 'Electricity_generated'}, inplace=True)
# Future = Future.reindex(columns=['Order','Operating_Date', 'Electricity_generated', 'MAPE', 'MAE'])

# display(Historical_data)
# display(Future)


# final_result = pd.concat([Historical_data, Future])
# final_result = final_result.fillna(0) # Fill NaN values with zero
# final_result['Electricity_generated'] = final_result['Electricity_generated'].astype(int)
# final_result['MAPE'] = final_result['MAPE'].astype(int)
# final_result['MAE'] = final_result['MAE'].astype(int)


# display(final_result)

# # Write Back the results into the lakehouse
# table_name = "Demand_Forecast_New_1"
# spark.createDataFrame(final_result).write.mode("overwrite").format("delta").save(f"Tables/{table_name}")
# print(f"Spark dataframe saved to delta table: {table_name}")
